In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("mbd-prj").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/05 10:50:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Reading the datasets

In [23]:
from pathlib import Path
lymsys_folder = Path("/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Big Data/Project/data/lmsys-chat-1m")
wildchat_folder = Path("/Users/lgk1910/Library/CloudStorage/OneDrive-AaltoUniversity/Learning2/Big Data/Project/data/WildChat-1M-Full/WildChat-1M-Full/")

In [24]:
# test reading the dataset from a folder of parquet files
lymsys_df = spark.read.parquet(str(lymsys_folder))
lymsys_df.show(5)
wildchat_df = spark.read.parquet(str(wildchat_folder))
wildchat_df.show(5)


+--------------------+----------+-----------------------------------+----+----------+--------------------+--------+
|     conversation_id|     model|                       conversation|turn|  language|   openai_moderation|redacted|
+--------------------+----------+-----------------------------------+----+----------+--------------------+--------+
|c20948192699451b8...|vicuna-13b|               [{Given the artic...|   1|   English|[{{false, false, ...|    true|
|8b02aa2df44847a1b...|vicuna-33b|               [{Alright, let's ...|   2|   English|[{{false, false, ...|    true|
|c18f3612fac140cb9...|vicuna-13b|[{あなたが最終更新されたのはいつ...|   5|  Japanese|[{{false, false, ...|   false|
|d1fabb62e3364665a...| llama-13b|               [{You are an AI a...|   1|   English|[{{false, false, ...|    true|
|b2da335248f04e439...| koala-13b|               [{Reescreva esse ...|   1|Portuguese|[{{false, false, ...|   false|
+--------------------+----------+-----------------------------------+----+----------+--

# Sample codes

In [25]:
# take the column 'conversation' from the first row of the dataframe, convert it to a list
lym_conv_sample = lymsys_df.select('conversation').first().conversation
print(lym_conv_sample[0].asDict().keys())
wild_conv_sample = wildchat_df.select('conversation').first().conversation
print(wild_conv_sample[0].asDict().keys())

dict_keys(['content', 'role'])
dict_keys(['content', 'country', 'hashed_ip', 'header', 'language', 'redacted', 'role', 'state', 'timestamp', 'toxic', 'turn_identifier'])


In [32]:
wildchat_df.select('header').first().header


Row(accept-language='en-US,en;q=0.9', user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36')

In [31]:
# search all rows with hashed_ip = 'f96c8515aa663db23ee086f36b7d3642036d5394bbbeada3fe0c6c8285f4d314'
wild_ip = 'f96c8515aa663db23ee086f36b7d3642036d5394bbbeada3fe0c6c8285f4d314'
wild_ip_df = wildchat_df.filter(col('hashed_ip') == wild_ip)
wild_ip_df.show(5)


+--------------------+------------------+-------------------+--------------------+----+--------+--------------------+--------------------+-----+--------+--------+-------------+--------------------+--------------------+
|   conversation_hash|             model|          timestamp|        conversation|turn|language|   openai_moderation| detoxify_moderation|toxic|redacted|   state|      country|           hashed_ip|              header|
+--------------------+------------------+-------------------+--------------------+----+--------+--------------------+--------------------+-----+--------+--------+-------------+--------------------+--------------------+
|f01a345e668b978b6...|gpt-3.5-turbo-0613|2023-09-13 04:14:27|[{(In the clubroo...|   1| English|[{{false, false, ...|[{2.1957975695841...|false|   false|Michigan|United States|f96c8515aa663db23...|{en-US,en;q=0.9, ...|
|136971e6080d62e86...|gpt-3.5-turbo-0613|2023-09-13 09:40:52|[{(In the clubroo...|   1| English|[{{false, false, ...|[{2.195